<a href="https://colab.research.google.com/github/guzonghua/saavlabs/blob/main/Gu_Highway_DQN_rlagents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training a DQN on `highway-v0` with rlagents
## Import requirements

In [ ]:
# Environment
#!pip install highway-env
!pip install git+https://github.com/guzonghua/highway-env
import gym
import highway_env

# Agent
!pip install git+https://github.com/eleurent/rl-agents

# Visualisation utils
import sys
%load_ext tensorboard
!pip install tensorboardx gym pyvirtualdisplay
!apt-get install -y xvfb python-opengl ffmpeg
!git clone https://github.com/eleurent/highway-env.git
sys.path.insert(0, '/content/highway-env/scripts/')
from utils import show_videos

  Cloning https://github.com/guzonghua/highway-env to /tmp/pip-req-build-7pafdait
  Running command git clone -q https://github.com/guzonghua/highway-env /tmp/pip-req-build-7pafdait
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for highway-env: filename=highway_env-1.2-cp37-none-any.whl size=92262 sha256=500ff6ddaeb508fbc85a7d0e6b3caece378871ff083a75bb82c64f451447ce9a
  Stored in directory: /tmp/pip-ephem-wheel-cache-pc5k4tcj/wheels/de/82/c1/11b1b7cb3ff8b817cf2892697fd3dab1b15739c269b87fe630
Successfully built highway-env
  Cloning https://github.com/eleurent/rl-agents to /tmp/pip-req-build-t9d09rah
  Running command git clone -q https://github.com/eleurent/rl-agents /tmp/pip-req-build-t9d09rah
  Created wheel for rl-agents: filename=rl_agents-1.0.dev0-cp37-none-any.whl size=114548 sha256=3295c7e431a043f32b470a0b841d79e389f6da60e5fa6db3601e063e74abc

## Training

Prepare environment, agent, and evaluation process.


In [ ]:
from rl_agents.trainer.evaluation import Evaluation
from rl_agents.agents.common.factory import load_agent, load_environment

# Get the environment and agent configurations from the rl-agents repository
!git clone https://github.com/eleurent/rl-agents.git
%cd /content/rl-agents/scripts/
env_config = 'configs/HighwayEnv/env.json'
agent_config = 'configs/HighwayEnv/agents/DQNAgent/dqn.json'
#env_config = 'configs/IntersectionEnv/env.json'
#agent_config = 'configs/IntersectionEnv/agents/DQNAgent/ego_attention_2h.json'

env = load_environment(env_config)
import pprint
from matplotlib import pyplot as plt
env.config["vehicles_count"] = 0
#env.config['reward_speed_range'] = [0, 100]
env.config["lanes_count"] = 2
env.config["initial_lane_id"] = 0
pprint.pprint(env.config)
pprint.pprint(agent_config)

env.reset()
plt.imshow(env.render(mode="rgb_array"))
plt.show()

agent = load_agent(agent_config, env)
evaluation = Evaluation(env, agent, num_episodes=10, display_env=False)
print(f"Ready to train {agent} on {env}")

fatal: destination path 'rl-agents' already exists and is not an empty directory.
/content/rl-agents/scripts
INFO: Making new env: highway-v0
{'action': {'type': 'DiscreteMetaAction'},
 'centering_position': [0.3, 0.5],
 'collision_reward': -1,
 'controlled_vehicles': 1,
 'duration': 40,
 'ego_spacing': 2,
 'id': 'highway-v0',
 'import_module': 'highway_env',
 'initial_lane_id': 0,
 'lanes_count': 2,
 'manual_control': False,
 'observation': {'type': 'Kinematics'},
 'offroad_terminal': False,
 'offscreen_rendering': False,
 'other_vehicles_type': 'highway_env.vehicle.behavior.IDMVehicle',
 'policy_frequency': 1,
 'real_time_rendering': False,
 'render_agent': True,
 'reward_speed_range': [20, 30],
 'scaling': 5.5,
 'screen_height': 150,
 'screen_width': 600,
 'show_trajectories': False,
 'simulation_frequency': 15,
 'vehicles_count': 0,
 'vehicles_density': 1}
'configs/HighwayEnv/agents/DQNAgent/dqn.json'


error: ignored

Start training. Run tensorboard locally to visualize training.

In [ ]:
#%tensorboard --logdir "{evaluation.directory}"
evaluation.train()

Progress can be visualised in the tensorboard cell above, which should update every 30s (or manually). You may need to click the *Fit domain to data* buttons below each graph.

## Testing

Run the learned policy for a few episodes.

In [ ]:
#env = load_environment(env_config)
env.configure({"offscreen_rendering": True})
agent = load_agent(agent_config, env)
evaluation = Evaluation(env, agent, num_episodes=3, recover=True)
evaluation.test()
show_videos(evaluation.run_directory)